In [80]:
from pipeline.generate_tool_schema_bedrock import generate_json_schema
from gen_ai_hub.proxy.native.amazon.clients import Session

model_name = 'anthropic--claude-3-opus'
bedrock = Session().client(model_name=model_name)

def call_bedrock(messages, tool_list):
    bedrock = Session().client(model_name=model_name)
    response = bedrock.converse(
        messages = messages, 
        inferenceConfig = {
            "maxTokens": 4096, 
            "temperature": 0.0, 
        },
        toolConfig={
            "tools": tool_list
        }
    )
    return response

In [81]:
################################
# Define Your Tool functions Here

def cosine(x: int) -> float:
    """
    Calculate the cosine of an integer 
    
    :param x: The input integer
    :return: A float representing the cosine of x
    """
    import math
    return math.cos(x)

def sin(x: int) -> float:
    """
    Calculate the sine of an integer

    :param x: The input integer
    :return: A float representing the sine of x
    """
    import math
    return math.sin(x)

def addition(x: float, y: float) -> float:
    """
    Calculate the addition of two float numbers

    :param x: The input float x
    :param y: The input float y
    :return: A float representing the sum of x and y
    """
    return x + y

In [90]:
def run_loop(prompt, tool_list, function_mapping):
    import json
    continue_loop = True
    
    messages = []
    user_message =  {"role": "user", "content": [{'text': prompt}]}
    messages.append(user_message)

    while continue_loop: 
        for line in messages:
            print(line)
        print('\n')
        response = call_bedrock( 
            messages = messages,
            tool_list=tool_list)
        
        response_message = response['output']['message']
        
        if response['stopReason'] == "end_turn":
            return response_message['content']

        response_content_blocks = response_message['content']
        messages.append({"role": "assistant", "content": response_content_blocks})
        follow_up_content_blocks = []
        for content_block in response_content_blocks:
                
            if 'toolUse' in content_block:
                tool_use_block = content_block['toolUse']
                tool_use_id = tool_use_block['toolUseId']
                tool_use_name = tool_use_block['name']
                tool_use_input = tool_use_block['input']
                tool_call_function = function_mapping.get(tool_use_name)
                tool_call_result = tool_call_function(**tool_use_input)
            
                follow_up_content_blocks.append({
                        "toolResult": {
                            'toolUseId': tool_use_id,
                            'content':[
                                {
                                    "json": {
                                        "result": tool_call_result
                                    }
                                }
                            ]
                        }
                    })
        
        if len(follow_up_content_blocks) > 0:
            function_call_result_message  = {
                 "role" : "user", 
                 "content": follow_up_content_blocks
            }
            messages.append(function_call_result_message) 

In [91]:
tool_list = []
tool_list.append(generate_json_schema(cosine))
tool_list.append(generate_json_schema(sin))
tool_list.append(generate_json_schema(addition))

function_mapping = {
    "cosine": cosine, 
    "sin": sin,
    "addition": addition
}

run_loop(prompt = "Calculate cos(3) + sin(4) for me", 
         tool_list = tool_list, 
         function_mapping = function_mapping)

{'role': 'user', 'content': [{'text': 'Calculate cos(3) + sin(4) for me'}]}


{'role': 'user', 'content': [{'text': 'Calculate cos(3) + sin(4) for me'}]}
{'role': 'assistant', 'content': [{'text': '<thinking>\nTo answer this request, we will need to use two tools:\n1. The cosine tool to calculate cos(3). It requires an integer parameter x, which is provided by the user (x=3).\n2. The sin tool to calculate sin(4). It also requires an integer parameter x, which is provided by the user (x=4). \n3. To get the final result, we will need to add the outputs of the cosine and sin tools. The addition tool takes two float parameters x and y. We can get these values from the outputs of the cosine and sin tools.\n\nSo the order of operations will be:\n1. Call cosine with x=3\n2. Call sin with x=4 \n3. Call addition with x=output of cosine and y=output of sin\n</thinking>'}, {'toolUse': {'toolUseId': 'tooluse_PekI2-VeRZ-We9h0MLL1XA', 'name': 'cosine', 'input': {'x': 3}}}, {'toolUse': {'toolUseId': 

[{'text': 'Therefore, cos(3) + sin(4) = -1.7467949919083736'}]